# Building agents with langchain

Basics on agents
* In agents, a language model is used as a reasoning engine to determine which actions to take and in which order. This is different from langchain "chains" where the steps taken are hardcoded. 
* Agents work with tools. You can define various tools like websearch or RAG which the agent can then call.
* In langchain, you create an agent like `from langchain.agents import create_tool_calling_agent`, however the agent doesn't execute those steps; that's done by `AgentExecutor`.
* By default, agents don't have access to chat history, but this can be setup. 

In [28]:
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, chat
from langchain_core.tools import Tool
from langchain.tools import format_tool_to_openai_function

import os
from dotenv import load_dotenv

load_dotenv()

True

## Defining tools
In langchain, "tools" are functions that the agent can use to perform specific tasks. Each tool is defined with three main components:
1. A name (e.g., SearchDocs, GenerateCode, GenerateDeckState)
2. A function that implements the tool's functionality
3. A description of what the tool does

Tool use can be nested: A tool like GenerateCode by call SearchDocs as part of its execution.

In [21]:
def search_docs(query: str) -> str:
    """Search the Opentrons documentation for relevant information."""
    docs = retriever.get_relevant_documents(query)
    return "\n".join(doc.page_content for doc in docs)

def generate_code(command: str) -> str:
    """Generate Python code for Opentrons based on the given command."""
    # First, we might want to search the documentation for relevant information
    docs = search_docs(command)
    
    # Then, we could use an LLM to generate the code based on the command and docs
    prompt = f"""
    Given the following command for an Opentrons robot:
    {command}
    
    And the following relevant documentation:
    {docs}
    
    Generate Python code to execute this command on an Opentrons robot. 
    Include necessary imports and assume the protocol object is already created.
    """
    
    response = llm.invoke(prompt)
    
    # You might want to add some post-processing or validation here
    
    return response

tools = [
    Tool(
        name="SearchDocs",
        func=search_docs,
        description="Useful for when you need to find information in the Opentrons documentation."
    ),
    Tool(
        name="GenerateCode",
        func=generate_code,
        description="Use this to generate Python code for Opentrons based on a command."
    )
]

## Create agent prompt

In [22]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant that helps translate natural language commands into Opentrons Python code. "
               "Use the provided tools to search the documentation and generate code."),
    ("human", "{input}"),
    chat.MessagesPlaceholder(variable_name="agent_scratchpad")
])

## Set up the LLM and create the agent

In [23]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [24]:
def process_command(user_input: str) -> str:
    response = agent_executor.invoke({"input": user_input})
    return response["output"]

In [25]:
commands = [
    "Set the temperature of a thermocycler block to 20 degC",
    "pipette 100 uL from well A1 to well B1 on a 96-well plate"
]

for command in commands:
    print(f"Command: {command}")
    result = process_command(command)
    print(f"Result:\n{result}\n")

Command: Set the temperature of a thermocycler block to 20 degC


> Entering new AgentExecutor chain...

Invoking: `GenerateCode` with `{'args': [20], 'config': {'tags': ['thermocycler', 'temperature'], 'run_name': 'Set Thermocycler Temperature'}}`




ToolException: Too many arguments to single-input tool GenerateCode.
                Consider using StructuredTool instead. Args: [[20], {'tags': ['thermocycler', 'temperature'], 'run_name': 'Set Thermocycler Temperature'}]

# Quick start example
https://python.langchain.com/v0.1/docs/modules/agents/quick_start/

Building an agent that can look things up online and retrieve info from some data of our own.

In [29]:
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv

load_dotenv()


True

## Create web search tool

In [30]:
search = TavilySearchResults()

In [32]:
search.invoke("How do I adjust the temperature of a thermocycler block on the Opentrons OT2 robot?")

[{'url': 'https://support.opentrons.com/s/article/Thermocycler-Module',
  'content': '1. Load your protocol, go to setup. 2. Select Module Controls > Thermocycler Module GEN2 > three-dot menu ( ⋮ ). Now you can select Set block temperature or Set lid temperature. Note: These options can also be accessed on the Devices page by clicking the three-dot menu ( ⋮) for the Thermocycler Module.'},
 {'url': 'https://support.opentrons.com/s/article/Protocol-Designer-Thermocycler-Module-steps',
  'content': 'Both Thermocycler state changes and profiles pause the robot as they occur. Changing the Thermocycler state. To change the hold state of the Thermocycler, build a Thermocycler step and select the "Change Thermocycler state" option. Using this step you can control: Thermocycler block temperature. Lid temperature. Lid position (open or closed)'},
 {'url': 'https://insights.opentrons.com/hubfs/Products/Modules/Thermocycler Manual.pdf',
  'content': 'ensure the sample block returns to idle temper

## Create a retriever over PDF docs
Retrieve Opentrons documentation

In [52]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("../rag-documentation/opentrons-docs.pdf")
documents = loader.load()

def clean_text(text):
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
    # Replace multiple newlines with a single newline
    text = re.sub(r'\n+', '\n', text)
    return text.strip()

cleaned_docs = [Document(page_content=clean_text(doc.page_content), metadata=doc.metadata) for doc in documents]
splits = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200
        ).split_documents(cleaned_docs)
vector = FAISS.from_documents(splits, OpenAIEmbeddings())
retriever = vector.as_retriever(search_kwargs={"k": 3})

In [53]:
splits

[Document(metadata={'source': '../rag-documentation/opentrons-docs.pdf', 'page': 0}, page_content='Introduction to Opentrons OT-2 for AI Assistants The Opentrons OT-2 is an open-source liquid handling robot designed for automated pipetting and protocol execution in life science laboratories. This document provides a concise overview of the key information needed to control an OT-2 robot using an AI language model. The goal is to enable AI assistants to understand OT-2 capabilities, constraints, and API commands in order to generate valid OT-2 protocols based on high-level user intent. The document covers: OT-2 hardware overview and specifications ● API basics and key concepts ● Pipettes and liquid handling ● Labware and deck setup ● Building block commands for aspirate, dispense, etc. ● Complex commands for common pipetting patterns ● Protocol examples demonstrating API usage By digesting this curated information, language models can engage in intelligent conversational protocol genera

In [54]:
retriever.invoke("How do I adjust the temperature of a thermocycler block on the Opentrons OT2 robot?")

[Document(metadata={'source': '../rag-documentation/opentrons-docs.pdf', 'page': 13}, page_content='The Thermocycler can control its block temperature, including holding at a temperature and adjusting for the volume of liquid held in its loaded plate. Temperature To set the block temperature inside the Thermocycler, use set_block_temperature(). At minimum you have to specify a temperature in degrees Celsius: tc_mod.set_block_temperature(temperature=4) If you don’t specify any other parameters, the Thermocycler will hold this temperature until a new temperature is set, deactivate_block() is called, or the module is powered off. New in version 2.0. Hold Time You can optionally instruct the Thermocycler to hold its block temperature for a specific amount of time. You can specify hold_time_minutes, hold_time_seconds, or both (in which case they will be added together). For example, this will set the block to 4 °C for 4 minutes and 15 seconds: tc_mod.set_block_temperature( temperature=4, ho

Let's turn the retriever into a tool that can be called by agents

In [59]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "opentrons_search",
    "Search for documentation about the Python Opentrons v2 API. For any questions about writing Opentrons code, you must use this tool!",
)

Now that we have created both tools, we can create a list of tools for the agent to access

In [60]:
tools = [search, retriever_tool]

Now that we have defined the tools, we can create the agent. We will be using an OpenAI Functions agent - for more information on this type of agent, as well as other options, see [this guide](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/).

In [61]:
# Choose LLM for guiding agent

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [82]:
from langchain import hub
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.prompts import HumanMessagePromptTemplate, MessagesPlaceholder

# Get the prompt to use - you can modify this!
default_prompt = hub.pull("hwchase17/openai-functions-agent")
default_prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [81]:

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=("You are an AI assistant that helps translate natural language commands into Opentrons Python code. Use the provided tools to search the documentation and generate code.")),
    MessagesPlaceholder(variable_name='chat_history', optional=True),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

prompt.messages

[SystemMessage(content='You are an AI assistant that helps translate natural language commands into Opentrons Python code. Use the provided tools to search the documentation and generate code.'),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [78]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [79]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [80]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "How do I adjust the temperature of a thermocycler block on the Opentrons OT2 robot?"}, chat_history=[])



> Entering new AgentExecutor chain...

Invoking: `opentrons_search` with `{'query': 'adjust temperature thermocycler block Opentrons OT2'}`


The Thermocycler can control its block temperature, including holding at a temperature and adjusting for the volume of liquid held in its loaded plate. Temperature To set the block temperature inside the Thermocycler, use set_block_temperature(). At minimum you have to specify a temperature in degrees Celsius: tc_mod.set_block_temperature(temperature=4) If you don’t specify any other parameters, the Thermocycler will hold this temperature until a new temperature is set, deactivate_block() is called, or the module is powered off. New in version 2.0. Hold Time You can optionally instruct the Thermocycler to hold its block temperature for a specific amount of time. You can specify hold_time_minutes, hold_time_seconds, or both (in which case they will be added together). For example, this will set the block to 4 °C for 4 minutes and 15 seconds: tc_

{'input': 'How do I adjust the temperature of a thermocycler block on the Opentrons OT2 robot?',
 'output': "To adjust the temperature of a thermocycler block on the Opentrons OT2 robot, you can use the `set_block_temperature()` method. Here’s how you can do it:\n\n1. **Set the Temperature**: You need to specify the temperature in degrees Celsius.\n2. **Optional Hold Time**: You can also specify how long to hold that temperature using `hold_time_minutes` and/or `hold_time_seconds`.\n\nHere’s an example code snippet that sets the thermocycler block to 4 °C for 4 minutes and 15 seconds:\n\n```python\n# Assuming you have already imported the necessary modules and initialized the protocol\nfrom opentrons import protocol_api\n\n# Create a protocol context\nprotocol = protocol_api.ProtocolContext()\n\n# Load the thermocycler module\ntc_mod = protocol.load_module('thermocycler')\n\n# Set the block temperature to 4 °C for 4 minutes and 15 seconds\ntc_mod.set_block_temperature(temperature=4, ho

If we want to keep track of chat history automatically, we can wrap this in a `RunnableWithMessageHistory`. For more information on how to use this, see [this guide](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).

In [84]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

In [85]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [86]:
agent_with_chat_history.invoke(
    {"input": "How do I adjust the temperature of a thermocycler block on the Opentrons OT2 robot?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `opentrons_search` with `{'query': 'adjust temperature thermocycler block Opentrons OT2'}`


The Thermocycler can control its block temperature, including holding at a temperature and adjusting for the volume of liquid held in its loaded plate. Temperature To set the block temperature inside the Thermocycler, use set_block_temperature(). At minimum you have to specify a temperature in degrees Celsius: tc_mod.set_block_temperature(temperature=4) If you don’t specify any other parameters, the Thermocycler will hold this temperature until a new temperature is set, deactivate_block() is called, or the module is powered off. New in version 2.0. Hold Time You can optionally instruct the Thermocycler to hold its block temperature for a specific amount of time. You can specify hold_time_minutes, hold_time_seconds, or both (in which case they will be added together). For example, this will set the block to 4 °C for 4 minutes and 15 seconds: tc_

{'input': 'How do I adjust the temperature of a thermocycler block on the Opentrons OT2 robot?',
 'chat_history': [],
 'output': "To adjust the temperature of a thermocycler block on the Opentrons OT2 robot, you can use the `set_block_temperature()` method. Here’s how you can do it:\n\n1. **Set the Temperature**: You need to specify the temperature in degrees Celsius.\n2. **Optional Hold Time**: You can also specify how long to hold that temperature using `hold_time_minutes` and/or `hold_time_seconds`.\n\nHere’s an example code snippet that sets the thermocycler block to 4 °C for 4 minutes and 15 seconds:\n\n```python\n# Assuming you have already imported the necessary modules and initialized the protocol\nfrom opentrons import protocol_api\n\n# Create a protocol context\nprotocol = protocol_api.ProtocolContext()\n\n# Load the thermocycler module\ntc_mod = protocol.load_module('thermocycler')\n\n# Set the block temperature to 4 °C for 4 minutes and 15 seconds\ntc_mod.set_block_temperat

In [87]:
agent_with_chat_history.invoke(
    {"input": "Provide just the code needed"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
```python
from opentrons import protocol_api

# Create a protocol context
protocol = protocol_api.ProtocolContext()

# Load the thermocycler module
tc_mod = protocol.load_module('thermocycler')

# Set the block temperature to 4 °C for 4 minutes and 15 seconds
tc_mod.set_block_temperature(temperature=4, hold_time_minutes=4, hold_time_seconds=15)
```

> Finished chain.


{'input': 'Provide just the code needed',
 'chat_history': [HumanMessage(content='How do I adjust the temperature of a thermocycler block on the Opentrons OT2 robot?'),
  AIMessage(content="To adjust the temperature of a thermocycler block on the Opentrons OT2 robot, you can use the `set_block_temperature()` method. Here’s how you can do it:\n\n1. **Set the Temperature**: You need to specify the temperature in degrees Celsius.\n2. **Optional Hold Time**: You can also specify how long to hold that temperature using `hold_time_minutes` and/or `hold_time_seconds`.\n\nHere’s an example code snippet that sets the thermocycler block to 4 °C for 4 minutes and 15 seconds:\n\n```python\n# Assuming you have already imported the necessary modules and initialized the protocol\nfrom opentrons import protocol_api\n\n# Create a protocol context\nprotocol = protocol_api.ProtocolContext()\n\n# Load the thermocycler module\ntc_mod = protocol.load_module('thermocycler')\n\n# Set the block temperature to 